<a href="https://colab.research.google.com/github/kareemullah1234/AI_Agent_content/blob/main/Agent_wiki_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install groq httpx simpleeval -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 3.7 MB/s eta 0:00:00


In [4]:
from simpleeval import SimpleEval

def calculate(expr):
    expr = ''.join(c for c in expr if c in '0123456789+-*/(). ')
    expr = expr.strip()
    if not expr:
        return "Error: Empty expression"
    try:
        return str(SimpleEval().eval(expr))
    except SyntaxError:
        return "Error: Invalid expression"
    except NameError:
        return "Error: Contains invalid variables"
    except ZeroDivisionError:
        return "Error: Division by zero"
    except Exception as e:
        return f"Error: {e}"

# Test it
print(calculate("15 * 25"))   # → "375"
print(calculate("1 / 0"))     # → "Error: Division by zero"
print(calculate("1 + xyz"))   # → "Error: Contains invalid variables"

375
Error: Division by zero
Error: Invalid expression


In [9]:
# !/usr/bin/env python
# coding: utf-8

# # 🧠 ReAct Agent with Groq + Wikipedia (Corrected & Robust)

import re
import httpx
from groq import Groq
#from simpleeval import SimpleEval, ParseError, NumberTooHigh

# 🔑 Hardcoded Groq API Key


# Initialize Groq client
client = Groq(api_key=GROQ_API_KEY)

# ========================
# 🤖 Agent Class
# ========================
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        try:
            completion = client.chat.completions.create(
                model="llama3-8b-8192",  # Fast & capable
                temperature=0,
                max_tokens=512,
                messages=self.messages
            )
            return completion.choices[0].message.content
        except Exception as e:
            return f"API Error: {str(e)}"


# ========================
# 🔧 Tools (Actions)
# ========================

def calculate(expr):
    """Safely evaluate mathematical expressions."""
    expr = re.sub(r'[^0-9+\-*/().\s]', '', expr)  # Sanitize
    try:
        result = SimpleEval().eval(expr)
        return str(result)
    except NumberTooHigh:
        return "Error: Number too large"
    except ParseError:
        return "Error: Invalid expression"
    except Exception as e:
        return f"Error: {str(e)}"

def wikipedia(query):
    """Search Wikipedia and return the snippet of the top result."""
    try:
        response = httpx.get(
            "https://en.wikipedia.org/w/api.php",
            params={
                "action": "query",
                "list": "search",
                "srsearch": query,
                "format": "json"
            },
            timeout=10.0
        )
        data = response.json()
        if data["query"]["search"]:
            snippet = data["query"]["search"][0]["snippet"]
            snippet = re.sub(r'<[^>]+>', '', snippet)  # Remove HTML
            snippet = re.sub(r'\s+', ' ', snippet)     # Normalize spaces
            return snippet.strip()
        return "No Wikipedia results found."
    except Exception as e:
        return f"Wikipedia error: {str(e)}"

# Register known actions
known_actions = {
    "calculate": calculate,
    "wikipedia": wikipedia
}


# ========================
# 📜 ReAct Prompt (Strict Format)
# ========================
# prompt = """
# You run in a loop of Thought, Action, PAUSE, Observation.
# At the end of the loop you output an Answer.

# Use Thought to describe your reasoning.
# Use Action to run **one** of the available actions in this exact format:
# Action: action_name: input
# Then say PAUSE.

# You can only do **one thing at a time**. Never combine multiple actions.
# Never make up values — always use tools to look things up.

# Your available actions are:

# calculate:
# e.g. calculate: 4 * 7 / 3
# Evaluates a math expression using Python syntax.

# wikipedia:
# e.g. wikipedia: Python programming language
# Returns a summary from Wikipedia.

# Always prefer to use Wikipedia to look up factual information.

# Example:
# Question: What is the capital of France?
# Thought: I should look up France on Wikipedia
# Action: wikipedia: France
# PAUSE

# Observation: France is a country in Western Europe. The capital is Paris.
# Answer: The capital of France is Paris.
# """.strip()

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation, and Response.

At each step:
1. Use **Thought** to reason about what to do next.
2. Use **Action** to call one of the available functions in the exact format:
   Action: action_name: input
3. Then say **PAUSE**.
4. You will be given an **Observation** with the result.
5. Repeat until you have enough information.
6. Finally, respond with **Answer: ...**

### Rules:
- You must **only** use the actions listed below.
- You can only perform **one action per turn**.
- Never combine multiple actions in one step.
- Never make up facts — always use `wikipedia` to verify.
- Never use `Action:` to give your final answer.
- When you have enough information, output `Answer: ...` and stop.

### Available Actions:
calculate:
e.g. calculate: 4 * 7 / 3
Evaluates a mathematical expression using Python syntax.

wikipedia:
e.g. wikipedia: Mount Everest
Searches Wikipedia and returns a summary snippet.

### Example Session:

Question: What is the tallest mountain in the world?
Thought: I need to find the tallest mountain. I should search Wikipedia.
Action: wikipedia: tallest mountain in the world
PAUSE

Observation: Mount Everest is the highest mountain above sea level, located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China. It stands at 8,849 metres (29,032 ft).

Thought: Now I know the tallest mountain is Mount Everest and its height.
Answer: The tallest mountain in the world is Mount Everest, which stands at 8,849 meters (29,032 feet) above sea level.
""".strip()
# ========================
# 🔍 Action Parser
# ========================
action_re = re.compile(r'^Action: (\w+): (.+)$')

def extract_action(text):
    """Extract the first valid action from the response."""
    for line in text.strip().split('\n'):
        match = action_re.match(line.strip())
        if match:
            return match.groups()
    return None


# ========================
# 🔁 Main Query Loop
# ========================
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)

        # Extract action
        action_match = extract_action(result)
        if not action_match:
            break  # No valid action → likely final answer

        action, action_input = action_match
        if action not in known_actions:
            next_prompt = f"Error: Unknown action '{action}'. Use only 'calculate' or 'wikipedia'."
            print(f"Error: Unknown action: {action}")
            continue

        print(f" -- running {action}: {action_input}")
        observation = known_actions[action](action_input)
        print("Observation:", observation)
        next_prompt = f"Observation: {observation}"

    # Final message history (optional)
    # print("\nFull conversation:")
    # for msg in bot.messages:
    #     print(f"{msg['role']}: {msg['content']}\n")


# ========================
# 🧪 Test Cases
# ========================

In [ ]:
#MAths

In [10]:
query("Fifteen times twenty five?")

Thought: I need to calculate the result of the multiplication.
Action: calculate: 15 * 25
PAUSE
 -- running calculate: 15 * 25
Observation: 375
Thought: Now I know the result of the multiplication.
Action: None
PAUSE


In [7]:
#Geography (Uses Wikipedia)

In [11]:
query("What is the tallest mountain in the world?")

Thought: I need to find the tallest mountain. I should search Wikipedia.

Action: wikipedia: tallest mountain in the world

PAUSE
 -- running wikipedia: tallest mountain in the world
Observation: This is a list of mountain peaks ordered by their topographic prominence. The prominence of a peak is the minimum height of climb to the summit on any route
Thought: That's not what I was looking for. I need to know the name of the tallest mountain. I should try again.

Action: wikipedia: Mount Everest

PAUSE
 -- running wikipedia: Mount Everest
Observation: misplaced vowels or missing conjuncts instead of Tibetan characters. Mount Everest (known locally as Sagarmatha in Nepal and Qomolangma in Tibet), is Earth&#039;s
Thought: Ah, I think I'm getting closer! Mount Everest is the name of the tallest mountain. I should try to find its height.

Action: wikipedia: Mount Everest

PAUSE
 -- running wikipedia: Mount Everest
Observation: misplaced vowels or missing conjuncts instead of Tibetan charact